# Overview

This tutorial is about loading an Ollama model and interacting with it. This interaction includes messages. The tutorial ends by makeing a simple chat application that keeps chat history.

# Load a chat model

Chat models can generate text by continuing a given prompt. First we need to load a model that has been pulled via Ollama (see README.md for details).

In [12]:
from langchain_ollama import ChatOllama

In [13]:
# Initialize the ChatOllama model with the specified model name
model_name = 'qwen3-vl:4b'

# and initialize the ChatOllama instance
chat_model = ChatOllama(
    model=model_name,
    validate_model_on_init=True,
    temperature=0.7
)

## Model invocation

We can invoke a model, i.e., give it a prompt to generate output, either by passing a string of the prompt, or by providing messages. Messages are either in the form of dictionaries or special LangChain classes

### Invocation with direct text

We can invoke the message with a simple string that includes the prompt.

In [14]:
# Invoke model with string input
response = chat_model.invoke("How can I make a delicious chocolate cake?")

In [15]:
# The response is an AIMessage object that includes the model's reply
print(type(response))

# We can get the content of the response like this
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
Making a **delicious, moist, and perfectly baked chocolate cake** is all about balancing the right ingredients, technique, and timing. Here’s a **tested, reliable recipe** (with *why* each step matters) that’s beginner-friendly and guaranteed to impress—no fancy tools needed!

---

### 🍫 **The Ultimate Chocolate Cake Recipe**  
*(Serves 8–10 | Prep: 25 mins | Bake: 35–40 mins)*  

#### **Why This Works**  
- **High-quality chocolate** (semi-sweet or dark, 60–70% cacao) adds depth.  
- **Room temperature ingredients** = smoother batter, better texture.  
- **No overmixing** = tender crumb (overmixing = tough cake).  
- **Butter + sugar creaming** = air pockets for lightness.  
- **Baking powder/salt** = balanced rise and flavor.  

---

### 📋 **Ingredients**  
*(Adjust for dietary needs: see "Pro Tips" below)*  

| **Ingredient** | **Amount** | **Why?** |
|----------------|------------|----------|
| **Butter** (unsalted) | 1 cup (225g) | *R

### Invocation with LangChain messages

We can invoke a message with LangChain messages. These messages are objects of classes, like the `AIMessage`, that are intepreted by models as having different roles within the chat, affecting how the model parses them differently.

Along with the `AIMessage` class, there is a `SystemMessage` and a `HumanMessage class`. There is also a `TollMessage` class, but we will see it in a next tutorial.

- `SystemMessage` - Tells the model how to behave and provide context for interactions.
- `HumanMessage` - Represents user input and interactions with the model.
- `AIMessage` - Responses generated by the model, including text content, tool calls, and metadata.
- `ToolMessge` - Represents the outputs of tool calls (next tutorial).

For more information on messages you can visit: https://docs.langchain.com/oss/python/langchain/messages#tool-message

In [16]:
# Let's initialize a system message and a human message
from langchain.messages import SystemMessage, HumanMessage
system_message = SystemMessage(content="You are a helpful assistant that provides cooking tips.")
human_message = HumanMessage(content="Can you suggest a recipe for chocolate cake?")

messages = [system_message, human_message]

# Now, we can pass these messages to the chat model
response = chat_model.invoke(messages)

In [17]:
# The response is again an AIMessage object
print(type(response))

# And we can print the content of the response
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
Absolutely! Here's a **classic, foolproof chocolate cake recipe** with tips for perfect texture and flavor. It’s simple, uses pantry staples, and works for beginners (no fancy tools needed). I’ve included **key pro tips** to avoid common mistakes.

---

### 🍫 **Rich Chocolate Cake**  
*(Serves 8–10; 10–12 minutes prep, 35–40 mins bake)*  
**Total time**: ~50 mins (plus cooling)

#### 🥣 **Ingredients**  
*(Serves 8–10)*  
- **2 cups (250g) all-purpose flour**  
- **1½ cups (300g) granulated sugar**  
- **¾ cup (150g) unsweetened cocoa powder** *(use Dutch-process for best flavor)*  
- **1½ tsp baking soda**  
- **1 tsp baking powder**  
- **1 tsp salt**  
- **1 cup (240ml) buttermilk** *(or 1 cup milk + 1 tbsp vinegar)*  
- **1 cup (240ml) vegetable oil** *(or melted shortening)*  
- **2 large eggs**, room temp  
- **1 tsp vanilla extract**  
- **1½ cups (360ml) hot water** *(key for moistness!)*  

#### 🧁 **For the Cake**  
*(Mix all ingre

### Continuning the chat

To make a chat application that keeps memory of the previous chats, we can append the generated `AIMessage` to the list of messages that we started with and then append new `HumanMessage`s after that. While this provides the full context to the LLM, after some messages the context is going to become so large that it will not fit the LLM. Additional steps will need to be made (e.g., summarization of previous chats, among others), but those are beyond the scope of this tutorial.

In [18]:
# Append the response to the messages list
messages.append(response)

# Get a new message from the user
new_human_message = HumanMessage(content="I would like a vegan version.")

messages.append(new_human_message)

# Invoke the model with the updated messages
response = chat_model.invoke(messages)

In [19]:
print(response.content)

Absolutely! Here's a **perfectly moist, rich vegan chocolate cake** that’s *just as delicious* as the original—no dairy, eggs, or animal products. I’ve optimized it for **flourless texture** (no added flour), **deep chocolate flavor**, and **easy vegan substitutions** (even if you’re new to baking). This recipe works for **all-vegan** diets and **gluten-free** if you use a gluten-free flour blend (though it’s not necessary here).

---

### 🌱 **Vegan Chocolate Cake (No Eggs, No Dairy, No Animal Products!)**  
*(Serves 8–10; 10 mins prep, 35–40 mins bake)*  
**Total time**: ~50 mins (plus cooling)  

#### 🥣 **Ingredients**  
*(Serves 8–10)*  
- **2 cups (250g) all-purpose flour** (or 1:1 gluten-free flour blend)  
- **1.5 cups (300g) granulated sugar**  
- **¾ cup (150g) unsweetened cocoa powder** *(Dutch-process for best flavor)*  
- **1.5 tsp baking soda**  
- **1 tsp baking powder**  
- **1 tsp salt**  
- **1 cup (240ml) unsweetened soy milk** *(or almond milk)*  
- **1 tbsp white vin

### Invocation with dictionaries

Except from LangChain messages, dictionaries can be used. Dictionaries have to be in the OpenAI chat completions format. These offer more flexibility with specific types of data, but are not well-suited for all models. For more information, please see:

https://platform.openai.com/docs/api-reference/chat/create

In [20]:
messages = [
    {"role": "system", "content": "You are a poetry expert"},
    {"role": "user", "content": "Write a haiku about spring"},
    {"role": "assistant", "content": "Cherry blossoms bloom..."}
]
response = chat_model.invoke(messages)

In [22]:
print(response.content)

  
Warm sun melts winter's last frost,  
New life stirs in soil.  

*Note: This haiku follows the traditional 5-7-5 syllable structure and captures the essence of spring through imagery of blooming cherry blossoms, the gentle warmth of the sun, and the renewal of life in the earth. The third line extends the theme with the quiet promise of growth and rebirth.*


## Streaming

All the above can be done using streamint, i.e., receiving each next word as it is generated by the model. In contrast to `invoke`, we now need to use the `stream` function of the model which returns an iterator.

For more information on streaming please see:

https://docs.langchain.com/oss/python/langchain/models#stream

In [27]:
# We start again with system and human messages
system_message = SystemMessage(content="You are a helpful assistant that provides cooking tips.")
human_message = HumanMessage(content="Can you suggest a recipe for chocolate cake?")

messages = [system_message, human_message]

# To keep the final response, we need to keep all the generated chunks as they stream in
full = None

for chunk in chat_model.stream(messages):
    print(chunk.text, end="|", flush=True)
    full = chunk if full is None else full + chunk

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||Abs

In [28]:
# The response is now an AIMessageChunk object
print(type(full))

# And we can print the content of the response
print(full.content_blocks)

<class 'langchain_core.messages.ai.AIMessageChunk'>
[{'type': 'text', 'text': "Absolutely! Here's a **classic, moist, and foolproof chocolate cake recipe** that’s perfect for beginners and bakers of all levels. This recipe uses simple ingredients and avoids common pitfalls (like overmixing or underbaking), so you’ll get a tender, rich cake every time.  \n\n---\n\n### 🍰 **Chocolate Cake**  \n*Serves 8–10 | Prep: 20 mins | Bake: 35 mins | Total: ~1 hour 15 mins*  \n\n#### 📋 **Ingredients**  \n*(All measurements are for a standard 8-inch round pan)*  \n\n- **Dry Ingredients**  \n  - 2 cups (250g) all-purpose flour  \n  - 1½ tsp baking powder  \n  - ½ tsp salt  \n  - 1½ cups (150g) Dutch-processed cocoa powder *(use high-quality, unsweetened cocoa for best flavor)*  \n  - 1 tsp vanilla extract  \n\n- **Wet Ingredients**  \n  - 1 cup (240ml) buttermilk *(or substitute: 1 cup milk + 1 tbsp vinegar, let sit 5 mins)*  \n  - 1 cup (240ml) granulated sugar  \n  - 1 cup (240ml) vegetable oil *(or